# AI21 Labs Structured Summarization

This project is about pulling out important details from long pieces of text, like chat
histories, court transcripts, and earnings calls. The challenge is to gather lots of
information from these texts and put it into a database, making it easier to analyze
all that data.

In this example, we leverage the NER dataset supreme court dataset to practice extracting data from
legal documents (Supreme Court Transcripts).

<img src="../architecture.png">

## Prerequisites

Install the package dependencies and import libraries.

In [2]:
%%capture

%pip install --upgrade transformers datasets jsonlines ai21 langchain

We will use both the AI21 Labs APIs and AI21 Labs models hosted on Amazon Bedrock.

In [3]:
import ai21 # Using AI21 Labs APIs

api_key = 'gAXhJl0FdF9SvRIdd90WYFoXBIk1uilx'

ai21.api_key = api_key

## Data

The NER dataset contains full transcriptions for all Supreme Court cases, which we will use AI21 to summarize and extract data from. 

First we get the data from Hugging Face. 

In [4]:
from datasets import load_dataset
dataset = load_dataset(
    "pasinit/scotus",
    split='train')

# list train, validation, and test datasets, schema, and record count
dataset

Dataset({
    features: ['id', 'text', 'decisionDirection', 'respondent', 'label_ids', 'labels'],
    num_rows: 7417
})


### Sample Case Text #1
We will use two datasets as an example, here we select one well known one: 

**U.S. v. Nixon (1974)**, In this case, the special prosecutor in the Watergate affair subpoenaed audio tapes of the Oval Office conversations. President Nixon refused to turn over the tapes, claiming exectutive priviledge. The Supreme Court rled that the defendents' rights to evidence outweighed the President'srigt to executive priviledge if national security was not compromised.

We will use this case (ID "1973-172-02") as an example to see how our AI21 Powered Summarization does.

In [16]:
# Get the Nixon Dataset ID
case_data_1 = [i for i in dataset if i["id"] == "1973-172-02"][0]
sample_text_1 = case_data_1['text']
print(sample_text_1[:1000]) # preview

United States Supreme Court
UNITED STATES v. NIXON(1974)
No. 73-1766
Argued: July 8, 1974Decided: July 24, 1974





[Footnote * Together with No. 73-1834, Nixon, President of the United States v. United States, also on certiorari before judgment to the same court.

 Following indictment alleging violation of federal statutes by certain staff members of the White House and political supporters of the President, the Special Prosecutor filed a motion under Fed. Rule Crim. Proc. 17 (c) for a subpoena duces tecum for the production before trial of certain tapes and documents relating to precisely identified conversations and meetings between the President and others. The President, claiming executive privilege, filed a motion to quash the subpoena. The District Court, after treating the subpoenaed material as presumptively privileged, concluded that the Special Prosecutor had made a sufficient showing to rebut the presumption and that the requirements of Rule 17 (c) had been satisfied. The

Like many of the case texts, this text exceeds the character/token limits of some AI21 API. We will use it to try some of the segmentation techniques. 

In [20]:
print("Character Count: {}".format(len(sample_text_1)))
print("Word Count: {}".format(len(sample_text_1.split(" "))))

Character Count: 67380
Word Count: 10751


### Sample Case Text #2
For our second sample, we can try this out with another well known case: 

**Brown v. Board of Education (1954)**: In this milestone decision, the Supreme Court ruled that separating children in public schools on the basis of race was unconstitutional. It signaled the end of legalized racial segregation in the schools of the United States, overruling the "separate but equal" principle set forth in the 1896 Plessy v. Ferguson case.

In [5]:
# Get the Brown v Board of Education Case
case_data_2 = [i for i in dataset if i["id"] == "1954-085-01"][0]
sample_text_2 = case_data_2['text']
print(sample_text_2[:5000]) # preview

United States Supreme Court
BROWN v. BOARD OF EDUCATION(1955)
No. 1
Argued: Decided: May 31, 1955


1. 
Racial discrimination in public education is unconstitutional, 
347 
U.S. 483, 497, and all provisions of federal, state or local law requiring or permitting such discrimination must yield to this principle. P. 298.

2. 
The judgments below (except that in the Delaware case) are reversed and the cases are remanded to the District Courts to take such proceedings and enter such orders and decrees consistent with this opinion as are necessary and proper to admit the parties to these cases to public schools on a racially nondiscriminatory basis with all deliberate speed. P. 301.

 (a) School authorities have the primary responsibility for elucidating, assessing and solving the varied local school problems which may require solution in fully implementing the governing constitutional principles. P. 299.


 (b) Courts will have to consider whether the action of school authorities constitute

Like many of the case texts, this text is within the character/token limits of all AI21 APIs. We will use it to try some of those APIs directly as an alternative.  

In [6]:
print("Character Count: {}".format(len(sample_text_2)))
print("Word Count: {}".format(len(sample_text_2.split(" "))))

Character Count: 15476
Word Count: 2462


## Segmentation and Segment Summarization

The court transcripts are long texts, often messy texts. We try with breaking down the data into coherent units, based on topics and lines. Here we use try three techniques: 1/ [AI21 Labs Segmentation API](https://docs.ai21.com/reference/text-segmentation-api-ref), 2/ LangChang [CharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/character_text_splitter), and for Case #2 which fits within limits, 3/[AI21 Labs Summarize by Segment API](https://docs.ai21.com/reference/summarize-by-segment-api-ref)

For techniques 1 and 2, we need to also summarize the segments. For technique 3, we will use AI21 Labs API to segment and summarize the segmenets in a single call. 

These segment summaries will be used in the next section for further summarization and information extraction.

In [45]:
print("Character Count: {}".format(len(sample_text_1)))
print("Word Count: {}".format(len(sample_text_1.split(" "))))

Character Count: 67380
Word Count: 10751


Note that most of the texts are beyond the 50K limit the [AI21 Summarize by Segment API](https://docs.ai21.com/reference/summarize-by-segment-api-ref), so we wanted to figure out if we could get a similar result with a summary of segments.

**In order to compare if we could get similar results with our segmentation, we will proceed with the example Case 2, Brown v. Board of Education. It is small enough to fit in the APIs, so we can compare and see if the segmeneted approaches can produce similar quality results. Note because of the size of other cases, only the segmented approach would work for the dataset at large.**

Let's look at it again to remind ourselves.

In [7]:
print(sample_text_2[:10000])

United States Supreme Court
BROWN v. BOARD OF EDUCATION(1955)
No. 1
Argued: Decided: May 31, 1955


1. 
Racial discrimination in public education is unconstitutional, 
347 
U.S. 483, 497, and all provisions of federal, state or local law requiring or permitting such discrimination must yield to this principle. P. 298.

2. 
The judgments below (except that in the Delaware case) are reversed and the cases are remanded to the District Courts to take such proceedings and enter such orders and decrees consistent with this opinion as are necessary and proper to admit the parties to these cases to public schools on a racially nondiscriminatory basis with all deliberate speed. P. 301.

 (a) School authorities have the primary responsibility for elucidating, assessing and solving the varied local school problems which may require solution in fully implementing the governing constitutional principles. P. 299.


 (b) Courts will have to consider whether the action of school authorities constitute

### Segmentation Technique 1 - AI21 Labs Segmentation API

We leverage LangChain to store the segments, as we suspect that the summaries will do better if we maintain memory of the segments that came before it for downstream summarization and analysis tasks.

In [8]:
from ai21 import Segmentation
response = ai21.Segmentation.execute(
    source=sample_text_2,
    sourceType="TEXT")

segments_1_case_2 = response['segments']
segments_1_case_2

[{'segmentText': 'United States Supreme Court\nBROWN v. BOARD OF EDUCATION(1955)', 'segmentType': 'normal_text_short'},
 {'segmentText': 'No. 1\nArgued: Decided: May 31, 1955\n\n\n1.', 'segmentType': 'non_english'},
 {'segmentText': 'Racial discrimination in public education is unconstitutional, \n347 \nU.S. 483, 497, and all provisions of federal, state or local law requiring or permitting such discrimination must yield to this principle.\n\nP. 298.', 'segmentType': 'normal_text_short'},
 {'segmentText': '2. The judgments below (except that in the Delaware case) are reversed and the cases are remanded to the District Courts to take such proceedings and enter such orders and decrees consistent with this opinion as are necessary and proper to admit the parties to these cases to public schools on a racially nondiscriminatory basis with all deliberate speed.', 'segmentType': 'normal_text'},
 {'segmentText': 'P. 301.\n\n(a) School authorities have the primary responsibility for elucidating

In [9]:
print(len(segments_1_case_2))

38


This returned many segments (177 for our sample case). It struggled on certain legal notes, classifying as "non-english". 

This may be difficult to summarize effectively, because it created so many segments. 

To attempt to summarize this, we will leverage LangChain and a call to AI21 Labs Jurassic Ultra model. We used a prompt template to encourage the model to leverage previous segments when building a summary. 

**[TODO] -- The below code summarizes plantiff's arguments, I think we can make it more general and then use the output to summarize specific use cases below. This will limit API calls.**

In [49]:
from langchain.chains import LLMChain
from langchain.docstore.document import Document
from langchain.llms import AI21
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from pprint import pprint
from time import perf_counter

prompt_template = """You are a judge analyzing the petitioner argument. Process the following text\
and return a markdown formatted list of the key facts supporting the petitioner's case:
{text}
Your answer should be in markdown list form.
Your focus is on the facts of the case.
Your response should look like this sample response:
<sample response>
- the defendant's girlfriend testifies that she was with the defendant at the time
- the expert testifies that the damage was consistent with a automobile impact
</sample response>\n
LIST OF PETITIONER FACTS:\n
- """
prompt = PromptTemplate.from_template(prompt_template)

llm = AI21(model="j2-jumbo-instruct",
            ai21_api_key=api_key,
            numResults=1,
            maxTokens=300,
            temperature=0.1,
            # topKReturn=0,
            topP=.1,
          )

refine_template = (
    "Your job is to produce a final listing of the facts supporting the petitioner's case.\n"
    "Your focus is on the facts of the case."
    "We have provided an existing list of facts up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing list of facts"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original list of facts with the addition of new facts.\n"
    "If the context isn't useful, return the original list of facts.\n"
    "Return your answer as a markdown list.\n"
    "Your response should look like this sample response:"
    "<sample response>"
    "- the defendant's girlfriend testifies that she was with the defendant at the time\n"
    "- the expert testifies that the damage was consistent with a automobile impact\n"
    "</sample response>"
    "LIST OF PLAINTIFF FACTS:\n"
    "- "
)

def response_to_documents(segments):

    Document(
        page_content="text",
        metadata={"source": "local"}
    )

    return [
        Document(page_content=it['segmentText']) for it in segments]

def extract_plaintiff_arguments(segments):
    tick = perf_counter()
    refine_prompt = PromptTemplate.from_template(refine_template)
    chain = load_summarize_chain(
        llm=llm,
        chain_type="refine",
        question_prompt=prompt,
        refine_prompt=refine_prompt,
        return_intermediate_steps=False,
        input_key="input_documents",
        output_key="output_text",
    )
    segment_docs = response_to_documents(segments)
    result = chain({"input_documents": segment_docs}, return_only_outputs=True)
    tock = perf_counter()
    elapsed = tock - tick
    print(f"Completed in {elapsed} seconds")
    return result


args = extract_plaintiff_arguments(segments_1_case_2)

pprint(args)

Completed in 221.272056642003 seconds
{'output_text': 'Thurgood Marshall, chief counsel for the NAACP, argued the '
                'case before the Supreme Court\n'
                '- The parties, the United States, and the States of Florida, '
                'North Carolina, Arkansas, Oklahoma, Maryland, and Texas filed '
                'briefs and participated in the oral argument\n'
                '- The judgment in the Delaware case - ordering the immediate '
                'admission of the plaintiffs to schools previously attended '
                'only by white children - is affirmed on the basis of the '
                'principles stated in our May 17, 1954, opinion, but the case '
                'is remanded to the Supreme Court of Delaware for such further '
                'proceedings as that Court may deem necessary in light of this '
                'opinion.\n'
                '- It is so ordered.\n'
                '- The cases coming to us from Kansas, South Ca

In [ ]:
pprint(args)

## Segmentation Technique 2 -- LangChain RecursiveCharacterTextSplitter

First we use the LangChain Text Splitter to split the text into chunks. This technique resulted in fewer chunks or segments compared to AI21 Segmentation API.

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split first text value using langchain
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 100,
    length_function = len,
    is_separator_regex = False,
)

chunks = text_splitter.create_documents([sample_text_2])

In [17]:
print(chunks[0])

page_content='United States Supreme Court\nBROWN v. BOARD OF EDUCATION(1955)\nNo. 1\nArgued: Decided: May 31, 1955\n\n\n1. \nRacial discrimination in public education is unconstitutional, \n347 \nU.S. 483, 497, and all provisions of federal, state or local law requiring or permitting such discrimination must yield to this principle. P. 298.\n\n2. \nThe judgments below (except that in the Delaware case) are reversed and the cases are remanded to the District Courts to take such proceedings and enter such orders and decrees consistent with this opinion as are necessary and proper to admit the parties to these cases to public schools on a racially nondiscriminatory basis with all deliberate speed. P. 301.\n\n (a) School authorities have the primary responsibility for elucidating, assessing and solving the varied local school problems which may require solution in fully implementing the governing constitutional principles. P. 299.\n\n\n (b) Courts will have to consider whether the action o

In [18]:
len(chunks)

9

## Summarize Segments
Now we will use AI21 Labs Ultra to summarize each chunk, and build a list of summaries. 

In [31]:
# summarize text using ai21 labs j2 ultra llm
def summarize_text(chunk):
    prompt_template = f"""You are an experienced legal scholar.
        Summarize the follow section of a court case document contained in the <document> tags below in a concise grammatically correct paragraph format.
        Write the summary so an average person can clearly understand the output.

        <document>
        {chunk}
        </document>"""
    
    response = ai21.Completion.execute(
        model="j2-ultra",
        prompt=prompt_template,
        maxTokens=200,
        temperature=0.2,
        stopSequences=["##"]
    )
    
    summary = response["completions"][0]["data"]["text"]
    
    return summary

In [32]:
%%time

# summarize each text chunk and add to list of summaries
summaries = []

for index, chunk in enumerate(chunks):
    summary = summarize_text(dict(chunk)["page_content"])
    summaries.append(f"""{summary.strip()}\n\n""")
    print(f"Summarizing chunk: {index + 1}", end="\r")

CPU times: user 315 ms, sys: 20 ms, total: 335 ms
Wall time: 19.7 s


In [33]:
for chunk_summary in summaries:
    print('\n----\n' + chunk_summary.strip())


----
The United States Supreme Court decided in Brown v. Board of Education that racial discrimination in public education is unconstitutional. The court reversed the judgments below (except in the Delaware case) and remanded the cases to the District Courts to take such proceedings and enter such orders and decrees consistent with this opinion as are necessary and proper to admit the parties to public schools on a racially nondiscriminatory basis with all deliberate speed. School authorities have the primary responsibility for elucidating, assessing and solving the varied local school problems which may require solution in fully implementing the governing constitutional principles.

----
In this case, the court is requiring the defendants to make a prompt and reasonable start toward full compliance with the ruling of the court. The burden rests on the defendants to establish that additional time is necessary in the public interest and is consistent with good faith compliance at the e

Let's see what the original chunks would look like for a comparison. 

In [24]:
%%time

# summarize each text chunk and add to list of summaries
summaries_segments_1_case_2 = []

for index, segment in enumerate(segments_1_case_2):
    if segment['segmentType'] != 'non_english':
        summary = summarize_text(segment["segmentText"])
        summaries_segments_1_case_2.append(f"""{summary.strip()}\n\n""")
    else:
        summaries_segments_1_case_2.append(segment["segmentText"])
    print(f"Summarizing chunk: {index + 1}", end="\r")

CPU times: user 1.05 s, sys: 62.7 ms, total: 1.11 s
Wall time: 48.2 s


In [25]:
for segment_summary in summaries_segments_1_case_2:
    print('\n----\n' + segment_summary.strip())


----
This case was one of the most important in the history of the Court.
Brown v. Board of Education was a landmark case in the United States that declared that segregation in schools was unconstitutional. This decision helped to pave the way for the civil rights movement and the eventual desegregation of all public facilities.



----
No. 1
Argued: Decided: May 31, 1955


1.

----
Racial discrimination in public education is unconstitutional, and all provisions of federal, state, or local laws requiring or permitting such discrimination must yield to this principle. This was decided in the case "Brown v. Board of Education", which was decided on 347 U.S. 483, 497.



----
The court cases below (except the one in Delaware) are reversed, and the cases will be remanded to the District Courts. These courts will need to take proceedings and enter orders and decrees consistent with this opinion, which are necessary and proper to admit the parties involved to public schools on a racially n

## Summarization and Analysis

Here try out varous data summarization tasks, leveraging AI21 Labs Jurassic Ultra model. 

## Summarization of Segment Summaries

In [36]:
# summarize a series of individual summaries using ai21 labs j2 ultra llm
def summarize_summaries(summaries):
    prompt_template = f"""You are an experienced legal scholar.
        Summarize the follow court case document contained in the <document> tags below in a grammatically correct format.
        Write the summary in five sentences so an average person can clearly understand the output.

        <document>
        {summaries}
        </document>"""
    
    response = ai21.Completion.execute(
        model="j2-ultra",
        prompt=prompt_template,
        maxTokens=1000,
        temperature=0.5,
        stopSequences=["##"]
    )
    
    summary = response["completions"][0]["data"]["text"]
    
    return summary.strip()

## Summarize the LangChain Segment Summaries

In [37]:
%%time

# concat a single string of all chunk summaries
list_of_summaries = ""

for summary in summaries:
    list_of_summaries += summary

# create an overall summary
summary_of_summaries = summarize_summaries(list_of_summaries)

print(summary_of_summaries)

In Brown v. Board of Education, the Supreme Court ruled that segregation in schools is unconstitutional. The court also ruled that the cases will still be under the court's jurisdiction during the transition process. The court invited the Attorney General of the United States and the Attorneys General of all states to present their views. The court will consider problems related to administration, arising from the physical condition of the school plant, the school transportation system, personnel, revision of school districts and attendance areas, and revision of local laws and regulations.
CPU times: user 37.7 ms, sys: 3.35 ms, total: 41.1 ms
Wall time: 2.05 s


## Summarize the AI21 Segment Summaries

In [43]:
%%time

# concat a single string of all chunk summaries
list_of_summaries_2 = ""

for summary in summaries_segments_1_case_2:
    list_of_summaries_2 += summary.strip('\n')

# create an overall summary
summary_of_summaries_2 = summarize_summaries(list_of_summaries_2)

print(summary_of_summaries_2)

The court case document discusses a court case called "Brown v. Board of Education", which was one of the most important in the history of the Court. The case declared that segregation in schools was unconstitutional and helped to pave the way for the civil rights movement. The court remanded the cases to the District Courts, which were instructed to take proceedings and enter orders and decrees consistent with this opinion, which are necessary and proper to admit the parties involved to public schools on a racially nondiscriminatory basis, with all deliberate speed. The court maintained jurisdiction over these cases during the transition period.
CPU times: user 42.5 ms, sys: 3.42 ms, total: 45.9 ms
Wall time: 2.94 s


In [44]:
summary_of_summaries_2

'The court case document discusses a court case called "Brown v. Board of Education", which was one of the most important in the history of the Court. The case declared that segregation in schools was unconstitutional and helped to pave the way for the civil rights movement. The court remanded the cases to the District Courts, which were instructed to take proceedings and enter orders and decrees consistent with this opinion, which are necessary and proper to admit the parties involved to public schools on a racially nondiscriminatory basis, with all deliberate speed. The court maintained jurisdiction over these cases during the transition period.'

## Use AI21 Summarize API

In [48]:
import json

def summarize(text, focus=None):
    response = ai21.Summarize.execute(
        source=text,
        sourceType="TEXT",
        focus=focus
    )
    return response['summary']

print(summarize(sample_text_2, 'decision'))

The judgments below are reversed and the cases are remanded to the District Courts.
The courts will have to consider whether the action of school authorities constitutes good faith implementation of the governing constitutional principles, and will be guided by equitable principles.
The judgment in the Delaware case, ordering the immediate admission of the plaintiffs to schools previously attended only by white children, is affirmed, but the case is remanded to the Supreme Court of Delaware for further proceedings.
CHIEF JUSTICE WARREN delivered the opinion of the Court in these cases, declaring that racial discrimination in public education is unconstitutional.
In view of the nationwide importance of the decision, the Court invited the Attorney General of the United States and the Attorneys General of all states requiring or permitting racial discrimination in public education to present their views on the question.
The courts which originally heard these cases can best perform this j

## We also iterated using Sumarize (with focus) and iterated over them with the full case text

- **plaintiff**: The parties to these cases are remanded to the District Courts to take further proceedings.
The plaintiffs' personal interest in admission to public schools is at stake, and the courts will be guided by equitable principles in fashioning and effecting the decrees.
The courts will consider the defendants' plans to meet these problems.
The Court affirmed the judgment in the Delaware case, ordering the immediate admission of the plaintiffs to schools previously attended only by white children, and remanded the case to the Supreme Court of Delaware for further proceedings.
Harold R. Fatzer, Attorney General of Kansas, and Paul E. Wilson, Assistant Attorney General, argued the cause for appellees.
The Court invited the Attorney General of the United States and the Attorneys General of all states requiring or permitting racial discrimination in public education to present their views on the nationwide importance of the decision.
The plaintiffs in the Delaware case are ordered to be admitted to schools previously attended only by white children.

- **respondant arguments**: Robert L. Carter argued the cause for appellants in No. 1, Spottswood W. Robinson, III argued the cause for petitioners in Nos. 2 and 3, George E. C. Hayes argued the cause for petitioners in No. 4, and Louis L. Redding argued the cause for respondents in No. 5.
Thurgood Marshall argued the causes for appellants in Nos. 1, 2 and 3, petitioners in No. 4 and respondents in No. 5.
Harold R. Fatzer, Attorney General of Kansas, argued the cause for appellees in No. 1.
J. Lindsay Almond, Jr., Archibald G. Robertson, T. Justin Moore, John W. Riely, T. Justin Moore, Jr., Milton D. Korman, Vernon E. West, Chester H. Gray and Lyman J. Umstead argued the cause for respondents.
The parties, the United States, and the States of Florida, North Carolina, Arkansas, Oklahoma, Maryland, and Texas presented their views on the question of whether racial discrimination in public schools should be eliminated.

- **outcome**: The Court invited the Attorney General of the United States and the Attorneys General of all states requiring or permitting racial discrimination in public education to present their views on the question. Substantial progress has already been made in the District of Columbia and in Kansas and Delaware.

- **judgement**: The judgments below are reversed and the cases are remanded to the District Courts.
The courts will consider whether the action of school authorities constitutes good faith implementation of the governing constitutional principles, and will be guided by equitable principles.
Courts of equity may consider problems related to administration, physical condition of the school plant, school transportation system, personnel, revision of school districts and attendance areas into compact units, and revision of local laws and regulations.
The courts will retain jurisdiction of these cases during the period of transition.
The judgment in the Delaware case, ordering the immediate admission of the plaintiffs to schools previously attended only by white children, is affirmed, but the case is remanded to the Supreme Court of Delaware for further proceedings.
Thurgood Marshall argued the causes for appellants in Nos. 1, 2 and 3, petitioners in No. 4 and respondents in No. 5.
J. Lindsay Almond, Jr., Archibald G. Robertson, Milton D. Korman, Vernon E. West, Chester H. Gray and Lyman J. Umstead argued the cause for appellees, and Joseph Donald Craven argued the cause for petitioners.
CHIEF JUSTICE WARREN delivered the opinion of the Court, declaring that racial discrimination in public education is unconstitutional.
In view of the nationwide importance of the decision, the Court invited the Attorney General of the United States and the Attorneys General of all states requiring or permitting racial discrimination in public education to present their views on the question.
The courts should remand the cases to the courts that originally heard the cases to consider whether the school authorities' actions constitute good faith implementation of the constitutional principles.
In fashioning and effectuating the decrees, the courts will be guided by equitable principles, and will take into account the public interest in eliminating obstacles to admission to public schools.
The courts will consider the adequacy of any plans the defendants may propose to meet the problems related to administration, the physical condition of the school plant, the school transportation system, personnel, revision of school districts and attendance areas into compact units, and revision of local laws and regulations.
The judgments below are reversed and the cases are remanded to the District Courts.
The judgment in the Delaware case is affirmed, but the case is remanded to the Supreme Court of Delaware for further proceedings.

- **dissenting judgement**: The dissenting judgements are reversed and the cases are remanded to the District Courts.
The Court affirmed the judgment in the Delaware case, but remanded the case to the Supreme Court of Delaware for further proceedings.
The courts will be guided by equitable principles in fashioning and effectuating the decrees, and will take into account the public interest in eliminating obstacles to admission to public schools.
The dissenting judgements are reversed and the cases are remanded to the District Courts.

- **footnotes**: The judgment in the Delaware case is affirmed, but the case is remanded to the Supreme Court of Delaware for further proceedings.

# Conclusion

1. Summarizing from a list of segements is challenging. For overall analysis, it is helpful for the model to have the entire context. Though prompt engineering and segment size.
2. Challenges: Legal writing and references. Segmentation struggled with those references. Is this a case for fine-tuning, or simple regex?
3. Techniques will be based on your different goals. 

For example: 

**Rag**  

- Who is the plaintiff?
- Who is the defendant?
- What is this case about?
- Why is this case important?
- Summary of the judgement
- Summary of dissenting argument
  
**Summarization**  

*Map Reduce*  

- Summarize the plaintiffs argument
- Summarize the defendants argument
  

*Refine*

- Extract the list of key points in the plaintiffs argument
- Extract the list of key points in the defendants argument
- Identify uncontested facts (occur in both arguments)
- Identify contested facts (conflicts between the arguments)